<a href="https://colab.research.google.com/github/beyza720/nlphw2/blob/main/orientation_multilanguage_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.utils import resample


drive.mount('/content/drive')

import pandas as pd

train_path = '/content/drive/My Drive/Colab Notebooks/orientation-lv-train.tsv'

train_orientation_data = pd.read_csv(train_path, sep='\t')


# Class imbalance for orientation
class_0 = train_orientation_data[train_orientation_data['label'] == 0]
class_1 = train_orientation_data[train_orientation_data['label'] == 1]

class_0_oversampled = resample(class_0, replace=True, n_samples=len(class_1), random_state=42)

balanced_train_orientation_data = pd.concat([class_0_oversampled, class_1])


# split the train data into 0.9 train and 0.1 validation
train_orientation, val_orientation = train_test_split(
    balanced_train_orientation_data,
    test_size=0.1,
    stratify=balanced_train_orientation_data['label'],
    random_state=42
)

print("Eğitim Veri Sayısı:", len(balanced_train_orientation_data))

print("Orientation Eğitim Seti:", len(train_orientation))
print("Orientation Validation Seti:", len(val_orientation))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Eğitim Veri Sayısı: 1256
Orientation Eğitim Seti: 1130
Orientation Validation Seti: 126


In [5]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [6]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = AutoModelForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased", num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_orientation)
val_dataset = Dataset.from_pandas(val_orientation)

def tokenize_function(examples):
    return tokenizer(examples["text_en"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

# print(tokenized_train_dataset[0])
# print(tokenized_train_dataset[1])

# from collections import Counter
# print(Counter(train_dataset["label"]))
# print(Counter(val_dataset["label"]))

Map:   0%|          | 0/1130 [00:00<?, ? examples/s]

Map:   0%|          | 0/126 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    report_to="none",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=71,
    save_steps=500,
    save_total_limit=2
)


In [10]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.3 MB/s eta 0:00:00


In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


In [13]:
from transformers import Trainer

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.678000,0.670697,0.626984,0.683673,0.626984,0.595796
2,0.510800,0.458438,0.817460,0.836538,0.817460,0.814836
3,0.340600,0.316895,0.880952,0.883367,0.880952,0.880765


TrainOutput(global_step=213, training_loss=0.5098047928071358, metrics={'train_runtime': 75.4102, 'train_samples_per_second': 44.954, 'train_steps_per_second': 2.825, 'total_flos': 891946477670400.0, 'train_loss': 0.5098047928071358, 'epoch': 3.0})